In [5]:
import wget

In [132]:
# url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py"
# wget.download(url)

In [133]:
#Docoment.json download
# url="https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json"
# wget.download(url)

In [ ]:
import minsearch

: 

In [7]:
import json

In [8]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [ ]:
docs_raw

In [9]:
documents=[]

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)


In [ ]:
documents

In [ ]:
documents[0]

Index the Document

In [10]:
index =minsearch.Index(
    text_fields=["question","text","section"],
    keyword_fields=["course"]

)

In [11]:
index.fit(documents)


In [12]:
q='the course has already started, can i still enroll?'

In [13]:
boost={'question':3.0,'section':0.5}

results=index.search(
    query=q,
    filter_dict={'course':'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

In [ ]:
results

In [146]:
from openai import OpenAI

In [147]:
from dotenv import load_dotenv
import os
from openai import OpenAI

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")




In [148]:
client = OpenAI(api_key=api_key)

In [149]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":q}]
)

In [ ]:
response.choices[0].message.content

In [151]:
prompt_template="""
You're a course teaching assistance. Answer the QUESTION based on the CONTEXT from the FAQ Databalse. Use only the facts from the CONTEXT when answering the QUESTION.
if the CONTEXT doesn't contain the answer, output NONE

Question :{question}

CONTEXT:
{context}
""".strip()

In [152]:
context = ""
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [ ]:
print(context)

In [154]:
prompt=prompt_template.format(question=q,context=context).strip()

In [156]:
response=client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role":"user","content":prompt}]
)

In [157]:
response.choices[0].message.content

"Yes, you can still enroll after the course has started. Even if you don't register on time, you are eligible to submit the homework. However, be mindful of the deadlines for final projects."

In [14]:
def search(query):
    boost={'question':3.0,'section':0.5}

    results=index.search(
        query=query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
)
    return results